In [1]:
import numpy as np
import pandas as pd
base_dir='/home/zxs/data/kaggle/dog_and_cat/'
all_feat = np.load(base_dir+'feature_vgg/train_feature.npy')
all_name = np.load(base_dir+'feature_vgg/train_filename.npy')
print all_feat.shape
print all_name.shape

Y = [1 if 'dog' in a else 0 for a in all_name]
Y = np.array(Y)
print Y.shape

(25024, 512)
(25024,)
(25024,)


In [2]:
import sys
sys.path.append('/home/zxs/LightGBM/python-package/')
import lightgbm as lgb

In [3]:
train_data = lgb.Dataset( all_feat, label=Y)

In [4]:
param = {'num_leaves':31, 'num_trees':100, 'objective':'binary' }
param['metric'] = 'binary_logloss'
num_round = 100
bst = lgb.train(param, train_data, num_round )

In [5]:
test_feat = np.load(base_dir+'feature_vgg/test_feature_ap.npy')
test_name = np.load(base_dir+'feature_vgg/test_filename.npy')
print test_feat.shape
print test_name.shape

(12512, 512)
(12512,)


In [7]:
pred_probs = bst.predict(test_feat)

In [10]:
pred_probs[:2]

array([ 0.01917277,  0.01138626], dtype=float32)

In [11]:
dogs = pred_probs
print test_name[:3]
print dogs[:3]

['2030.jpg' '2031.jpg' '2032.jpg']
[  1.91727672e-02   1.13862641e-02   7.29261374e-05]


In [12]:
name2label = dict(zip(test_name,dogs))
idx = list(xrange(1,12501))
labels = [name2label[str(id_)+'.jpg'] for id_ in idx]
d = {'id':np.array(idx),'label':np.array(labels)}
df = pd.DataFrame(data=d)
df.head()

,id,label
0,1,0.999845
1,2,0.999926
2,3,0.999890
3,4,0.999721
4,5,0.000177


In [13]:
df.to_csv('lgb_submit_all_1.csv',index=False)

# Esamble [有效]
- xgb (0.06338)
- lgb (0.06433)
- 单纯bagging 0.6068

In [16]:
xgb = pd.read_csv('xgb_submit_all_1.csv')
xgb.head()

,id,label
0,1,0.998601
1,2,0.999611
2,3,0.999197
3,4,0.999366
4,5,0.001720


In [17]:
xgb_dogs = xgb['label']

In [19]:
print xgb_dogs.shape
xgb_dogs[:5]

(12500,)


0    0.998601
1    0.999611
2    0.999197
3    0.999366
4    0.001720
Name: label, dtype: float64

In [21]:
lgb = pd.read_csv('lgb_submit_all_1.csv')
lgb.head()

,id,label
0,1,0.999845
1,2,0.999926
2,3,0.999890
3,4,0.999721
4,5,0.000177


In [22]:
lgb_dogs = lgb['label']
print lgb_dogs.shape
lgb_dogs[:5]

(12500,)


0    0.999845
1    0.999926
2    0.999890
3    0.999721
4    0.000177
Name: label, dtype: float64

In [23]:
bleed = (xgb_dogs+lgb_dogs)/2.

In [24]:
print bleed.shape
bleed[:5]

(12500,)


0    0.999223
1    0.999769
2    0.999543
3    0.999544
4    0.000948
Name: label, dtype: float64

In [26]:
idx = list(xrange(1,12501))
d = {'id':np.array(idx),'label':bleed}
df = pd.DataFrame(data=d)
df.head()
df.to_csv('bleed_lgb_xgb_submit_all_1.csv',index=False)

In [27]:
df.head()

,id,label
0,1,0.999223
1,2,0.999769
2,3,0.999543
3,4,0.999544
4,5,0.000948
